模型来自百度搜索自动生成的结果

https://www.baidu.com/s?ie=utf-8&f=8&rsv_bp=1&tn=baidu&wd=tensorflow%20%E5%85%89%E8%B0%B1%E5%88%86%E7%B1%BB&rn=20&oq=tensorflow%25E5%2592%258Cpytorch%25E5%2593%25AA%25E4%25B8%25AA%25E5%25A5%25BD&rsv_pq=ed842ba900410f29&rsv_t=97f1EiiG4GbyTHFucBBj6cK2ONxcudncNcI4%2B0NiHMnjJreOiUR6KhU9lQI&rqlang=cn&rsv_dl=tb&rsv_enter=1&rsv_btype=t&inputT=2430&rsv_sug3=45&rsv_sug2=0&prefixsug=tensorflow%2520%25E5%2585%2589%25E8%25B0%25B1%25E5%2588%2586%25E7%25B1%25BB&rsp=0&rsv_sug4=2509

In [1]:
# 导入常规包
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt

In [2]:
# 加载CNN相关库
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout
import tensorflow as tf

2024-05-07 18:45:00.847476: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-07 18:45:00.898507: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-07 18:45:00.899555: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-07 18:45:01.531306: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [11]:
from sklearn.model_selection import train_test_split

In [3]:
# 加载所有文件，使用生成式加载，最后得到的是 10x(10000x3000)的数组的列表
ori_data_l = [fits.getdata(f'train_data_{i+1:02d}.fits', 0) for i in range(10)]
labels_l = [fits.getdata(f'train_data_{i+1:02d}.fits', 1)["label"] for i in range(10)]

In [4]:
# 把数据串起来，注意方向
ori_data = np.vstack(ori_data_l)
labels = np.hstack(labels_l)
ori_data.shape, labels.shape

((100000, 3000), (100000,))

In [5]:
# 确认一下是有三种类型，以及每个类型有几条光谱
np.unique(labels), np.sum(labels==0), np.sum(labels==1), np.sum(labels==2)

(array([0, 1, 2]), 3427, 1011, 95562)

In [6]:
# 计算每颗星的最大值最小值，用于归一化
ori_max = np.max(ori_data, axis=1)
ori_min = np.min(ori_data, axis=1)

In [7]:
# 归一化。为避免出现0，干脆直接除以最大值
uni_data = np.empty_like(ori_data)
for i in range(ori_data.shape[0]):
    uni_data[i] = ori_data[i] / ori_max[i]

In [12]:
# 数据集合进行数组格式变换，变成(样本数，样本大小，1)，也就是(100000, 3000, 1)，强行给加一个维度
# 好像还有一个函数去增加一个维度，看到忘记了
sample_data = uni_data.reshape(uni_data.shape[0], ori_data.shape[1], 1)

In [9]:
# 按照网页给的，创建线性模型
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(sample_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [10]:
# 编译模型
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
# 拆分训练集和测试集
train_idx, test_idx = train_test_split(np.arange(labels.shape[0]), test_size=0.1)
train_data, train_labels = sample_data[train_idx], labels[train_idx]
test_data , test_labels  = sample_data[test_idx ], labels[test_idx ]

In [14]:
%%time
# 训练模型
model.fit(sample_train, labels, epochs=10, batch_size = 500,)

Epoch 1/10
200/200 [==============================] - 1s 5ms/step - loss: 0.1911 - accuracy: 0.9541
Epoch 2/10
200/200 [==============================] - 1s 4ms/step - loss: 0.0754 - accuracy: 0.9780
Epoch 3/10
200/200 [==============================] - 1s 4ms/step - loss: 0.0447 - accuracy: 0.9851
Epoch 4/10
200/200 [==============================] - 1s 4ms/step - loss: 0.0360 - accuracy: 0.9876
Epoch 5/10
200/200 [==============================] - 1s 4ms/step - loss: 0.0302 - accuracy: 0.9897
Epoch 6/10
200/200 [==============================] - 1s 6ms/step - loss: 0.0275 - accuracy: 0.9909
Epoch 7/10
200/200 [==============================] - 1s 5ms/step - loss: 0.0251 - accuracy: 0.9918
Epoch 8/10
200/200 [==============================] - 1s 4ms/step - loss: 0.0240 - accuracy: 0.9922
Epoch 9/10
200/200 [==============================] - 1s 5ms/step - loss: 0.0241 - accuracy: 0.9923
Epoch 10/10
200/200 [==============================] - 1s 4ms/step - loss: 0.0219 - accuracy: 0.9932

In [16]:
%%time
# 评估数据，就是用测试集
loss, accuracy = model.evaluate(test_data , test_labels)
print(loss, accuracy)

313/313 [==============================] - 0s 1ms/step - loss: 0.0194 - accuracy: 0.9941
0.019400639459490776 0.9940999746322632
CPU times: user 1.06 s, sys: 572 ms, total: 1.63 s
Wall time: 645 ms


In [19]:
# 读取测试数据，并且进行归一化、扩维
hdutest = fits.open('test_data.fits')
exam_data_0 = hdutest[0].data
exam_max = np.max(exam_data_0, axis=1)
exam_min = np.min(exam_data_0, axis=1)
exam_data = np.empty((*exam_data_0.shape, 1))
for i in range(exam_data_0.shape[0]):
    exam_data[i, :, 0] = exam_data_0[i] / exam_max[i]
exam_data.shape

(1000, 3000, 1)

In [20]:
%%time
# 对测试数据进行分类
exam_labels = model.predict(exam_data)

32/32 [==============================] - 0s 929us/step
CPU times: user 423 ms, sys: 186 ms, total: 609 ms
Wall time: 230 ms


In [21]:
exam_labels

array([[3.9116559e-05, 1.6757808e-06, 9.9995917e-01],
       [3.3664129e-08, 4.2027004e-09, 9.9999994e-01],
       [4.1923301e-10, 8.5738708e-12, 9.9999994e-01],
       ...,
       [4.9940649e-05, 7.0903488e-06, 9.9994296e-01],
       [3.5964746e-02, 3.1042736e-02, 9.3299246e-01],
       [4.4519785e-03, 2.7942962e-03, 9.9275374e-01]], dtype=float32)

In [22]:
# 从分类概率矩阵中选出每颗星最大可能性的编号
exam_class = np.argmax(exam_labels, axis=1)

In [24]:
exam_class

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 0, 0, 0, 0, 2, 2, 2,
       2, 2, 2, 0, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 0, 2, 2, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2,
       2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2,
       2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 1, 1,
       2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 2, 2, 2,
       2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0,
       2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2,
       0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 0, 2, 2, 2, 0, 2,
       0, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2,
       0, 2, 2, 1, 0, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1,
       2, 2, 0, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 1, 2, 0, 2, 2, 2, 0, 2, 2,
       2, 2, 2, 2, 2, 0, 0, 0, 2, 1, 2, 2, 0, 2, 0,